# import:

In [1]:
from __future__ import print_function
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *
from TCR_feature_generation.SequenceClusteringModule import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict, Counter
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

from Levenshtein import distance as levdist
from scipy.spatial.distance import pdist, squareform

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
PNP530Cardio126=PNP530+Cardio126


done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!
end of function!!!


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'12012019'

# generate interesting samplelists:

## old males

In [ ]:
f1='%s/TCR_real_data/predictions2/TargetDFs/PNP530Cardio126_AgeGender.xlsx' %MyPath
PNP530Cardio126_AgeGender=pd.read_excel(f1).set_index('BD')

print (PNP530Cardio126_AgeGender.shape)
PNP530Cardio126_AgeGender.head()


In [ ]:
PNP530Cardio126_AgeGender['cohort']=np.where((PNP530Cardio126_AgeGender.index.str.replace('BD','').astype(int)) > 949,1,0)
PNP530Cardio126_AgeGender.tail()

### check the desired age range to find groups with balanced age

In [ ]:
fig,ax=plt.subplots()
dataList=[]
groups= PNP530Cardio126_AgeGender.groupby('cohort')
for name,group in groups:
    tup=tuple([name,group['Age'].dropna().tolist() ])
    dataList.append(tup)
# print (dataList)
title=''

plotHistComprison(dataList,ax,title,showLegend=True,nBins=20,toAnnotate=True,alpha=None,plotType='hist')

plt.show()

In [ ]:
PNP530Cardio126_AgeGender_oldMales=PNP530Cardio126_AgeGender[(PNP530Cardio126_AgeGender['Gender_Male']==1) & (PNP530Cardio126_AgeGender['Age']>45) & (PNP530Cardio126_AgeGender['Age']<70)]

In [ ]:
fig,ax=plt.subplots()
dataList=[]
groups= PNP530Cardio126_AgeGender_oldMales.groupby('cohort')
for name,group in groups:
    tup=tuple([name,group['Age'].dropna().tolist() ])
    print (name, len(group['Age'].dropna().tolist()))
    dataList.append(tup)
# print (dataList)
title=''

plotHistComprison(dataList,ax,title,showLegend=True,nBins=20,toAnnotate=True,alpha=None,plotType='hist')

plt.show()

*** conclusion ***

when taking only males aged 45-70 (not including) - the age distributions are balanced according to ttest and KS test (57 samples for controls, 93 samples for patients, mean age controls=57.6
mean age patients= 59.4

### take out relatives:

In [ ]:
# get relative info:
f2='%s/Sample files/PNP530-relationships.xlsx' %MyPath
PNP530_relationships=pd.read_excel(f2)
print (PNP530_relationships.shape)
PNP530_relationships.head()

In [ ]:
#get all relatives (blood or household)
RelativeTable=PNP530_relationships[PNP530_relationships['is relative in PNP515?']==1].set_index('BD')
print (RelativeTable.shape)

In [ ]:
' BD121' in RelativeTable.Relative

#### define function:

the function drop_relatives was copied to SampleFileFunctions.py

In [ ]:
def drop_relatives(df,onlyBloodRels=False):
    
    #df index should be BD numbers!
    
    f2='%s/Sample files/PNP530-relationships.xlsx' %MyPath
    PNP530_relationships=pd.read_excel(f2)
    samplesToDrop=[]
    
    if onlyBloodRels:
        RelativeTable=PNP530_relationships[(PNP530_relationships['is relative in PNP515?']==1) & (PNP530_relationships['BloodGroup'].notnull())].set_index('BD')
    else:
        RelativeTable=PNP530_relationships[PNP530_relationships['is relative in PNP515?']==1].set_index('BD')

    for sample in df.index:
    #     print (sample[0])
        if sample in RelativeTable.index.tolist() and sample not in samplesToDrop:
            sample2=RelativeTable.loc[sample,'Relative']
            if sample2 in df.index.tolist():
#                 print ('dropping' , sample, sample2)
                samplesToDrop.append(sample2)

#     print (samplesToDrop)
    return samplesToDrop

#### check function on all samples:

In [ ]:
df=PNP530Cardio126_AgeGender
onlyBloodRels=False

print (len(PNP530Cardio126_AgeGender[PNP530Cardio126_AgeGender['cohort']==0]))

samplesToDrop=drop_relatives(df,onlyBloodRels)
          
print (len(samplesToDrop))

In [ ]:
df=PNP530Cardio126_AgeGender
onlyBloodRels=True

print (len(PNP530Cardio126_AgeGender[PNP530Cardio126_AgeGender['cohort']==0]))

samplesToDrop=drop_relatives(df,onlyBloodRels)
          
print (len(samplesToDrop))

#### check if there are relatives amond 'old_male' samples:
couldn't find relatives. no sample to drop!

In [ ]:
df=PNP530Cardio126_AgeGender_oldMales
onlyBloodRels=False

drop_relatives(df,onlyBloodRels)

### save old male samples:

In [ ]:


PNP530_balancedAge_males=PNP530Cardio126_AgeGender_oldMales[PNP530Cardio126_AgeGender_oldMales['cohort']==0].index.tolist()
print (len(PNP530_balancedAge_males))
print (PNP530_balancedAge_males[:5])

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530_balancedAge_males','wb') as fp:
    pickle.dump(PNP530_balancedAge_males,fp)


In [ ]:
Cardio126_balancedAge_males=PNP530Cardio126_AgeGender_oldMales[PNP530Cardio126_AgeGender_oldMales['cohort']==1].index.tolist()
print (len(Cardio126_balancedAge_males))
print (Cardio126_balancedAge_males[:5])

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_balancedAge_males','wb') as fp:
    pickle.dump(Cardio126_balancedAge_males,fp)

## top20 vs bottom20 for interesting continous phenotypes:

In [ ]:
phen_list=['GRACEScore','MaximalTroponin','Age']

for phen in phen_list:
    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/%s.xlsx' %phen
    phenDF=pd.read_excel(f1).set_index('BD').sort_values(by=phen)
    bottom20samples=phenDF[:20].index.tolist()
    top20samples=phenDF[-20:].index.tolist()
    
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_bottom20%s' %phen,'wb') as fp:
        pickle.dump(bottom20samples,fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_top20%s' %phen,'wb') as fp:
        pickle.dump(top20samples,fp)
    
    print (phen)
    print (len(bottom20samples))
    print ('max value of bottom group: ',phenDF.iloc[20,0] )
    print (len(top20samples))
    print ('min value of top group: ',phenDF.iloc[-20,0] )


## top30 vs bottom30 for interesting continous phenotypes:

In [ ]:
phen_list=['GRACEScore','MaximalTroponin','Age']

for phen in phen_list:
    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/%s.xlsx' %phen
    phenDF=pd.read_excel(f1).set_index('BD').sort_values(by=phen)
    bottom30samples=phenDF[:30].index.tolist()
    top30samples=phenDF[-30:].index.tolist()
    
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_bottom30%s' %phen,'wb') as fp:
        pickle.dump(bottom30samples,fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_top30%s' %phen,'wb') as fp:
        pickle.dump(top30samples,fp)
    
    print (phen)
    print (len(bottom30samples))
    print ('max value of bottom group: ',phenDF.iloc[30,0] )
    print (len(top30samples))
    print ('min value of top group: ',phenDF.iloc[-30,0] )


## ostial cx disease:

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/OstialCXDisease.xlsx'
OstialCXDisease=pd.read_excel(f1).set_index('BD')

OstialCXDisease.head()

ostial0samples=OstialCXDisease[OstialCXDisease['OstialCXDisease']==0].index.tolist()
ostial234samples=OstialCXDisease[OstialCXDisease['OstialCXDisease']>1].index.tolist()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/ostial0samples','wb') as fp:
    pickle.dump(ostial0samples,fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/ostial234samples' ,'wb') as fp:
    pickle.dump(ostial234samples,fp)

# print (len(ostial0samples))
# print (ostial0samples[:5])

# print (len(ostial234samples))
# print (ostial234samples[:5])

## binary phenotypes:

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Cardio126phenAllInfo.xlsx'
phenDF=pd.read_excel(f1).set_index('BD')

phenDF.columns.tolist()

In [ ]:
f2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Gender_Male.xlsx'
phenDF=phenDF.rename(columns={'Gender':'Gender_Male'})
phenDF['Gender_Male'].to_excel(f2)

In [ ]:
phen_list=['AdmissionDiagnosismapped_STEMI','ChiefComplaintmapped_anginalpain','DischargeClopidogrel','ChiefComplaintmapped_anginalpain','KnownCAD','Gender_Male']

for phen in phen_list:
    print (phen)

    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/%s.xlsx' %phen
    df=pd.read_excel(f1).set_index('BD')

    df.head()
    
    if phen=='Gender_Male':
        df0samples=df[df[phen]==2].index.tolist()
    else:
        df0samples=df[df[phen]==0].index.tolist()
    df1samples=df[df[phen]==1].index.tolist()

    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'wb') as fp:
        pickle.dump(df0samples,fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen ,'wb') as fp:
        pickle.dump(df1samples,fp)

    print (len(df0samples))
    print (df0samples[:5])

    print (len(df1samples))
    print (df1samples[:5])

## PCI_LVEF_CVA_CABG phenotypes:

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/PCI_LVEF_CVA_CABG.xlsx'
PCI_LVEF_CVA_CABG=pd.read_excel(f1).set_index('BD')

PCI_LVEF_CVA_CABG.columns.tolist()

In [ ]:
for phen in PCI_LVEF_CVA_CABG.columns.tolist():
    print (phen)

    df=pd.DataFrame(PCI_LVEF_CVA_CABG[phen])

    df.head()
    
    df0samples=df[df[phen]==0].index.tolist()
    df1samples=df[df[phen]==1].index.tolist()

    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'wb') as fp:
        pickle.dump(df0samples,fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen ,'wb') as fp:
        pickle.dump(df1samples,fp)

    print (len(df0samples))
    print (df0samples[:5])

    print (len(df1samples))
    print (df1samples[:5])

## PCI:

In [ ]:
PCI=pd.DataFrame(PCI_LVEF_CVA_CABG['PCI1_CABG_cont'])


PCI0samples=PCI[PCI['PCI1_CABG_cont']==0].index.tolist()
PCI2345samples=PCI[PCI['PCI1_CABG_cont']>1].index.tolist()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PCI0samples','wb') as fp:
    pickle.dump(PCI0samples,fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PCI2345samples' ,'wb') as fp:
    pickle.dump(PCI2345samples,fp)

print (len(PCI0samples))
print (PCI0samples[:5])

print (len(PCI2345samples))
print (PCI2345samples[:5])

## LVEF:

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/LVEF.xlsx'
LVEF=pd.read_excel(f1).set_index('BD')

LVEF.head()

LVEF0samples=LVEF[LVEF['LVEF']==0].index.tolist()
LVEF23samples=LVEF[(LVEF['LVEF']==2) |(LVEF['LVEF']==3) ].index.tolist()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/LVEF0samples','wb') as fp:
    pickle.dump(LVEF0samples,fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/LVEF23samples' ,'wb') as fp:
    pickle.dump(LVEF23samples,fp)

print (len(LVEF0samples))
print (LVEF0samples[:5])

print (len(LVEF23samples))
print (LVEF23samples[:5])

## GLUCOSE DISORDER

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/GlucoseDisorder.xlsx'
GlucoseDisorder=pd.read_excel(f1).set_index('BD')

GlucoseDisorder.head()

GlucoseDisorder0samples=GlucoseDisorder[GlucoseDisorder['GlucoseDisorder']==0].index.tolist()
GlucoseDisorder2samples=GlucoseDisorder[GlucoseDisorder['GlucoseDisorder']==2].index.tolist()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/GlucoseDisorder0samples','wb') as fp:
    pickle.dump(GlucoseDisorder0samples,fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/GlucoseDisorder2samples' ,'wb') as fp:
    pickle.dump(GlucoseDisorder2samples,fp)

print (len(GlucoseDisorder0samples))
print (GlucoseDisorder0samples[:5])

print (len(GlucoseDisorder2samples))
print (GlucoseDisorder2samples[:5])

## random sample list for the paper phenotypes:

### binary:

In [ ]:
phen_list=['ChiefComplaintmapped_anginalpain','DischargeClopidogrel','PCI1_CABG']
for phen in phen_list:
    print (phen)

    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/%s.xlsx' %phen
    df=pd.read_excel(f1).set_index('BD')

    df.head()
    
    if phen=='Gender_Male':
        df0samples=df[df[phen]==2].index.tolist()
    else:
        df0samples=df[df[phen]==0].index.tolist()
    df1samples=df[df[phen]==1].index.tolist()

    print ('df0 length:',len(df0samples))
#     print (df0samples[:5])

    print ('df1 length:',len(df1samples))
#     print (df1samples[:5])
    
    if len(df0samples)>len(df1samples):
        df0samples=random.sample(df0samples,len(df1samples))
    else:
        df1samples=random.sample(df1samples,len(df0samples))
        
    print ('new length if df0 samples:',len(df0samples) )
    print ('new length if df1 samples:',len(df1samples) )   
    
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples_reSampled2' %phen,'wb') as fp:
        pickle.dump(df0samples,fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples_reSampled2' %phen ,'wb') as fp:
        pickle.dump(df1samples,fp)
    

### ostial:

In [ ]:
phen_list=['OstialCXDisease']
for phen in phen_list:
    print (phen)

    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/%s.xlsx' %phen
    df=pd.read_excel(f1).set_index('BD')

    df.head()
    
    if phen=='Gender_Male':
        df0samples=df[df[phen]==2].index.tolist()
    else:
        df0samples=df[df[phen]==0].index.tolist()
    df1samples=df[df[phen]==1].index.tolist()

    print ('df0 length:',len(df0samples))
#     print (df0samples[:5])

    print ('df1 length:',len(df1samples))
#     print (df1samples[:5])
    
    if len(df0samples)>len(df1samples):
        df0samples=random.sample(df0samples,len(df1samples))
    else:
        df1samples=random.sample(df1samples,len(df0samples))
        
    print ('new length if df0 samples:',len(df0samples) )
    print ('new length if df1 samples:',len(df1samples) )   
    
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/ostial0samples_reSampled2','wb') as fp:
        pickle.dump(df0samples,fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/ostial234samples_reSampled2','wb') as fp:
        pickle.dump(df1samples,fp)
    

# use fisher test to compare non-productive sequence cluster frequencies in patients and controls

## get non-productive clusters and check them:

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allNonProd_maxdist1/sampleByClusterDF.dat'
sampleByClusterDF_nonProd=pd.read_pickle(f1)
print (sampleByClusterDF_nonProd.shape)
print (sampleByClusterDF_nonProd.iloc[:4,:4])

In [ ]:
print (sampleByClusterDF_nonProd.count().min())
print (sampleByClusterDF_nonProd.count().max())

In [ ]:
sampleByClusterDF_nonProd.loc[:,(sampleByClusterDF_nonProd.count()==65)].columns.tolist()

### get all sequences in a cluster for a cluster head

In [ ]:
# get all sequences in a cluster for a cluster head
levDistMatTotalNonProdFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allNonProd_maxdist1/levDistMatTotal.dat'
levDistMatTotalNonProd=pd.read_pickle(levDistMatTotalNonProdFile)
levDistMatTotalNonProd.head()

In [ ]:
clusterMatrix=levDistMatTotalNonProd.groupby(['seq1id','seq2id']).count()
clusterMatrix.head()

In [ ]:
#get seqID matrix:
f3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allNonProd_maxdist1/seqID_nonProd.dat'
seqID=pd.read_pickle(f3)
seqID.head()

In [ ]:
print ('replacing seqIDs with real sequences...')
clusterMatrix_2=clusterMatrix.reset_index()
clusterMatrix_seqs=pd.merge(clusterMatrix_2,pd.DataFrame(seqID),how='left',left_on='seq2id',right_index=True).rename(columns={'seq':'seq2'})
clusterMatrix_seqs=pd.merge(clusterMatrix_seqs,pd.DataFrame(seqID),how='left',left_on='seq1id',right_index=True).rename(columns={'seq':'seq1'})
# clusterMatrix_seqs=clusterMatrix_seqs.drop(['seq1id','seq2id'],axis=1)
clusterMatrix_seqs.head()

In [ ]:
clusterMatrix_seqs_groupByHeadID=clusterMatrix_seqs.groupby(['seq1id','seq1','seq2']).count()
clusterMatrix_seqs_groupByHeadID.head()

In [ ]:
def get_cluster_seqeunces_per_clusterheadID(head,isSeq=True):
    
    if isSeq: 
        level='seq1id'
        clusterMatrix_seqs_groupByHeadID=clusterMatrix_seqs.groupby(['seq1','seq1id','seq2']).count()
    else:
        level='seq1'
        clusterMatrix_seqs_groupByHeadID=clusterMatrix_seqs.groupby(['seq1id','seq1','seq2']).count()
        
    clusterhead=clusterMatrix_seqs_groupByHeadID.loc[head,:].index.get_level_values(level)[0]
    seqs=clusterMatrix_seqs_groupByHeadID.loc[head,:].index.get_level_values('seq2').tolist()
    
    return clusterhead, seqs

In [ ]:
print (get_cluster_seqeunces_per_clusterheadID(12.0,False))

In [ ]:
print (get_cluster_seqeunces_per_clusterheadID('CASS**ETQYF'))

In [ ]:
print (get_cluster_seqeunces_per_clusterheadID(81155.0,False))

## fisher test to compare nonProd clusters between patients and controls

In [ ]:
the function compare_seq_clusters_between_populations was copied to SequenceClusteringModule.py

### run for all PNP vs cardio - non prod clusters

In [ ]:
sampleByClusterDF=sampleByClusterDF_nonProd
sampleGroup1=PNP530
sampleGroup1name='PNP530'
sampleGroup2=Cardio126
sampleGroup2name='Cardio126'
output_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allNonProd_maxdist1/Fisher_MW_comparisons/'

results=compare_seq_clusters_between_populations(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,percShared=0.05)

### run for old males - non prod clusters

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530_balancedAge_males','rb') as fp:
    PNP530_balancedAge_males=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_balancedAge_males','rb') as fp:
    Cardio126_balancedAge_males=pickle.load(fp)

sampleByClusterDF=sampleByClusterDF_nonProd
sampleGroup1=PNP530_balancedAge_males
sampleGroup1name='PNP530_balancedAge_males'
sampleGroup2=Cardio126_balancedAge_males
sampleGroup2name='Cardio126_balancedAge_males'
output_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allNonProd_maxdist1/Fisher_MW_comparisons/'

results=compare_seq_clusters_between_populations(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,percShared=0.05)

### generate non-productive sequence matrix and run the analysis on it as well:

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/AllUniqueWithCounts'
AllUniqueWithCounts=pd.read_pickle(f1)
AllUniqueWithCounts_nonProd=AllUniqueWithCounts[AllUniqueWithCounts['prod_stat']==0]
sampleBySeqs_nonProd=AllUniqueWithCounts_nonProd[['prod_stat','Sample']].reset_index().rename(columns={'index':'seqeunce'}).groupby(['Sample','seqeunce'])['prod_stat'].count().unstack()
print (sampleBySeqs_nonProd.shape)
print (sampleBySeqs_nonProd.count().sort_values(ascending=False).head())

In [ ]:
print (sampleBySeqs_nonProd.count().sort_values(ascending=False).tail())

In [ ]:
sampleBySeqs_nonProd.info(memory_usage='deep')

#### run for all PNP vs all Cardio

In [ ]:
sampleByClusterDF=sampleBySeqs_nonProd

print 
nShared_all=math.ceil(len(sampleByClusterDF) * percShared)
print ('nShared_all= ',nShared_all)
all_percClusters=sampleByClusterDF.loc[:,sampleByClusterDF.count()>=nShared_all].columns.tolist()

In [ ]:
sampleByClusterDF=sampleBySeqs_nonProd
sampleGroup1=PNP530
sampleGroup1name='PNP530_seqs'
sampleGroup2=Cardio126
sampleGroup2name='Cardio126_seqs'
output_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allNonProd_maxdist1/Fisher_MW_comparisons/'
percShared=0.02

results=compare_seq_clusters_between_populations(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,percShared)

# compare productive single sequences:

## define wrapper function:

In [ ]:
def compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                 percShared=0.05):
    f1='%s/TCR_real_data/predictions2/TargetDFs/PNP530Cardio126_AgeGender.xlsx' %MyPath
    PNP530Cardio126_AgeGender=pd.read_excel(f1).set_index('BD')

    All=sampleGroup1+sampleGroup2

    #calculate fisher and man-whitnney:
    sampleByClusterDF=sampleByClusterDF.loc[All,:]

    print ('starting...')
    results=compare_seq_clusters_between_populations(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,percShared=0.25)

\

    # compare age distributions:
    Age_sampleGroup1=PNP530Cardio126_AgeGender.loc[sampleGroup1,'Age'].dropna().tolist()
    Age_sampleGroup2=PNP530Cardio126_AgeGender.loc[sampleGroup2,'Age'].dropna().tolist()
    dataList=[(sampleGroup1name,Age_sampleGroup1),(sampleGroup2name,Age_sampleGroup2)]
    
    
    fig,ax=plt.subplots()
    # print (dataList)
    title='%s vs. %s - Age distribution' %(sampleGroup1name,sampleGroup2name)
    plotHistComprison(dataList,ax,title,showLegend=True,nBins=20,toAnnotate=True,alpha=None,plotType='hist')
    plt.show()

    #compare gender distribution:
    Gender_sampleGroup1=Counter(PNP530Cardio126_AgeGender.loc[sampleGroup1,'Gender_Male'].dropna().tolist())
    Gender_sampleGroup2=Counter(PNP530Cardio126_AgeGender.loc[sampleGroup2,'Gender_Male'].dropna().tolist())
    
    GenderDF=pd.concat([pd.DataFrame(index=[sampleGroup1name],data=Gender_sampleGroup1),
                        pd.DataFrame(index=[sampleGroup2name],data=Gender_sampleGroup2)]).fillna(0)
    print (GenderDF)
    
    try:
        R_f, p_f = fisher_exact(GenderDF, alternative='two-sided')
        print ('Fisher r,p = ',R_f,p_f)  
    except:
        print ('couldnt execute fisher test for gender distributions')
    
    return


## generate productive sequence matrix and run the analysis on it as well:

In [ ]:
f1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/sharingAnalysis/AllUniqueWithCounts'
AllUniqueWithCounts=pd.read_pickle(f1)
AllUniqueWithCounts_prod=AllUniqueWithCounts[AllUniqueWithCounts['prod_stat']==1]
sampleBySeqs_prod=AllUniqueWithCounts_prod[['prod_stat','Sample']].reset_index().rename(columns={'index':'seqeunce'}).groupby(['Sample','seqeunce'])['prod_stat'].count().unstack()
print (sampleBySeqs_prod.shape)
print (sampleBySeqs_prod.count().sort_values(ascending=False).head())

In [ ]:
sampleBySeqs_prod.iloc[:4,:4]

In [ ]:
# sort to include only sequences shared by at least 5% of samples in each cohort or together:
print ('starting')
perc=0.05
percTooMany=0.85
# get clusters that appear in Xperc of samples in sampleGroup1:
sampleBySeqs_prod_group1 = sampleBySeqs_prod.loc[PNP530, :]
nShared_group1 = math.ceil(len(sampleBySeqs_prod_group1) * perc)
print ('nShared_group1= ', nShared_group1)
group1_percClusters = sampleBySeqs_prod_group1.loc[:, sampleBySeqs_prod_group1.count() >= nShared_group1].columns.tolist()
print ('n clusters shared by %s or more samples in group1= ' % nShared_group1, len(group1_percClusters))

# get clusters that appear in Xperc of samples in sampleGroup2:
sampleBySeqs_prod_group2 = sampleBySeqs_prod.loc[Cardio126, :]
nShared_group2 = math.ceil(len(sampleBySeqs_prod_group2) * perc)
print ('nShared_group2= ', nShared_group2)
group2_percClusters = sampleBySeqs_prod_group2.loc[:, sampleBySeqs_prod_group2.count() >= nShared_group2].columns.tolist()
print ('n clusters shared by %s or more samples in group2= ' % nShared_group2, len(group2_percClusters))

# get clusters that appear in Xperc of All samples:
nShared_all = math.ceil(len(sampleBySeqs_prod) * perc)
print ('nShared_all= ', nShared_all)
all_percClusters = sampleBySeqs_prod.loc[:, sampleBySeqs_prod.count() >= nShared_all].columns.tolist()
print ('n clusters shared by %s or more samples in all samples= ' % nShared_all, len(all_percClusters))
    
# concatenate lists:
ClusterListShared = list(set(group1_percClusters + group2_percClusters + all_percClusters))
print ('final number of clusters to use= ', len(ClusterListShared))

# filter table to have only those sequences
sampleBySeqs_prod_perc = sampleBySeqs_prod.loc[:, ClusterListShared]
print ('final sampleBySeqs_prod_perc shape is: ', sampleBySeqs_prod_perc.shape)
# filter table to not include clusters that appear in more than percTooMany of samples:
nTooMany = math.ceil(len(sampleBySeqs_prod) * percTooMany)
print ('filtering out clusters that appear in more than %s (%s) samples' % (nTooMany, percTooMany))
sampleBySeqs_prod_perc = sampleBySeqs_prod_perc.loc[:, sampleBySeqs_prod_perc.count() < nTooMany]
print ('final sampleBySeqs_prod_perc shape is: ', sampleBySeqs_prod_perc.shape)


f6 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/sharingAnalysis/prod_sharedBy5-85percPerCohort.dat'
sampleBySeqs_prod_perc.to_pickle(f6)

In [ ]:

f6 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/prod_sharedBy5-85percPerCohort.dat'
sampleBySeqs_prod_perc.to_pickle(f6)

In [ ]:
print (sampleBySeqs_prod.count().sort_values(ascending=False).tail())

In [ ]:
sampleBySeqs_prod.info(memory_usage='deep')

## run for old males pnp vs old males cardio - prod sequences

In [ ]:
# get samples lists:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530_balancedAge_males') as fp:
    PNP530_balancedAge_males=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_balancedAge_males') as fp:
    Cardio126_balancedAge_males=pickle.load(fp)

sampleGroup1=PNP530_balancedAge_males
sampleGroup1name='PNP530_balancedAge_males_prodSeqs'
sampleGroup2=Cardio126_balancedAge_males
sampleGroup2name='Cardio126_balancedAge_males_prodSeqs'
output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
sampleByClusterDF=sampleBySeqs_prod
percShared=0.1

compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                             percShared)

## run for continuous phenotypes:

In [ ]:
phen_list=['GRACEScore','MaximalTroponin','Age']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_bottom20%s' %phen,'rb') as fp:
        Cardio126_bottom20=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_top20%s' %phen,'rb') as fp:
        Cardio126_top20=pickle.load(fp)

        sampleGroup1=Cardio126_bottom20
        sampleGroup1name='Cardio126_bottom20%s' %phen
        sampleGroup2=Cardio126_top20
        sampleGroup2name='Cardio126_top20%s' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleBySeqs_prod
        percShared=0.25

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

## run for binary phenotypes:

In [ ]:
phen_list=['DischargeClopidogrel','ChiefComplaintmapped_anginalpain','KnownCAD','Gender_Male']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
        Cardio126_0samples=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
        Cardio126_1samples=pickle.load(fp)

        sampleGroup1=Cardio126_0samples
        sampleGroup1name='%s0samples' %phen
        sampleGroup2=Cardio126_1samples
        sampleGroup2name='%s1samples' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleBySeqs_prod
        percShared=0.25

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

## other phenotyps:

In [ ]:
phen_list=[('GlucoseDisorder',0,2),('LVEF',0,23),('ostial',0,234)]

for item in phen_list:
    phen=item[0]
    group1=str(item[1])
    group2=str(item[2])
    
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s%ssamples' %(phen,group1),'rb') as fp:
        group1data=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s%ssamples' %(phen,group2),'rb') as fp:
        group2data=pickle.load(fp)

        sampleGroup1=group1data
        sampleGroup1name='%s%ssamples' %(phen,group1)
        sampleGroup2=group2data
        sampleGroup2name='%s%ssamples' %(phen,group2)
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleBySeqs_prod
        percShared=0.25

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

# Use productive clusters (redundant!)

## get productive clusters dataframe:


In [ ]:
sampleByClusterDF=pd.read_pickle('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/sampleByClusterDF_cohortfiltering005-09perc_dropped.dat')

## PNP530 vs Cardio126

In [ ]:
# get samples lists:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126') as fp:
    Cardio126=pickle.load(fp)

sampleGroup1=PNP530
sampleGroup1name='PNP530_prodClusRed'
sampleGroup2=Cardio126
sampleGroup2name='Cardio126_prodClusRed'
output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
sampleByClusterDF=sampleByClusterDF
percShared=0.05
percTooMany=0.85

compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                             percShared,percTooMany)

## balanced old males: PNP530 vs Cardio126

In [ ]:
# get samples lists:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530_balancedAge_males') as fp:
    PNP530_balancedAge_males=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_balancedAge_males') as fp:
    Cardio126_balancedAge_males=pickle.load(fp)

sampleGroup1=PNP530_balancedAge_males
sampleGroup1name='PNP530_balancedAge_males_prodClusRed'
sampleGroup2=Cardio126_balancedAge_males
sampleGroup2name='Cardio126_balancedAge_males_prodClusRed'
output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
sampleByClusterDF=sampleByClusterDF
percShared=0.1
percTooMany=0.85

compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                             percShared,percTooMany)

## run for continuous phenotypes - bottom/top 20

In [ ]:
# phen_list=['GRACEScore','MaximalTroponin','Age']
phen_list=['MaximalTroponin']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_bottom20%s' %phen,'rb') as fp:
        Cardio126_bottom20=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_top20%s' %phen,'rb') as fp:
        Cardio126_top20=pickle.load(fp)

        sampleGroup1=Cardio126_bottom20
        sampleGroup1name='Cardio126_bottom20%s' %phen
        sampleGroup2=Cardio126_top20
        sampleGroup2name='Cardio126_top20%s' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25
        

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

## run for continuous phenotypes - bottom/top 30

In [ ]:
# phen_list=['GRACEScore','MaximalTroponin','Age']
phen_list=['MaximalTroponin']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_bottom30%s' %phen,'rb') as fp:
        Cardio126_bottom30=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126_top30%s' %phen,'rb') as fp:
        Cardio126_top30=pickle.load(fp)

        sampleGroup1=Cardio126_bottom30
        sampleGroup1name='Cardio126_bottom30%s' %phen
        sampleGroup2=Cardio126_top30
        sampleGroup2name='Cardio126_top30%s' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25
        

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

## run for binary phenotypes:

In [ ]:
phen_list=['DischargeClopidogrel','ChiefComplaintmapped_anginalpain','KnownCAD','Gender_Male']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
        Cardio126_0samples=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
        Cardio126_1samples=pickle.load(fp)

        sampleGroup1=Cardio126_0samples
        sampleGroup1name='%s0samples' %phen
        sampleGroup2=Cardio126_1samples
        sampleGroup2name='%s1samples' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

## run for  more binary phenotypes:

In [ ]:
phen_list=['AdmissionDiagnosismapped_STEMI','ChiefComplaintmapped_anginalpain']
for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
        Cardio126_0samples=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
        Cardio126_1samples=pickle.load(fp)

        sampleGroup1=Cardio126_0samples
        sampleGroup1name='%s0samples' %phen
        sampleGroup2=Cardio126_1samples
        sampleGroup2name='%s1samples' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)

In [ ]:
phen='ChiefComplaintmapped_anginalpain'


with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
    Cardio126_0samples=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
    Cardio126_1samples=pickle.load(fp)
    
sampleGroup1=Cardio126_0samples
sampleGroup1name='%s0samples' %phen
sampleGroup2=Cardio126_1samples
sampleGroup2name='%s1samples' %phen
        
MyPath = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF'   
f1 = '/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TargetDFs/PNP530Cardio126_AgeGender.xlsx' 
PNP530Cardio126_AgeGender = pd.read_excel(f1).set_index('BD')

Age_sampleGroup1 = PNP530Cardio126_AgeGender.loc[sampleGroup1, 'Age'].dropna().tolist()
Age_sampleGroup2 = PNP530Cardio126_AgeGender.loc[sampleGroup2, 'Age'].dropna().tolist()
dataList = [(sampleGroup1name, Age_sampleGroup1), (sampleGroup2name, Age_sampleGroup2)]

try:
    fig, ax = plt.subplots()
    # print (dataList)
    title = '%s vs. %s - Age distribution' % (sampleGroup1name, sampleGroup2name)
    plotHistComprison(dataList, ax, title, showLegend=True, nBins=20, toAnnotate=True, alpha=None, plotType='hist')
    plt.show()
except:
    print ('cant do plots, sorry')

# compare gender distribution:
Gender_sampleGroup1 = Counter(PNP530Cardio126_AgeGender.loc[sampleGroup1, 'Gender_Male'].dropna().tolist())
Gender_sampleGroup2 = Counter(PNP530Cardio126_AgeGender.loc[sampleGroup2, 'Gender_Male'].dropna().tolist())

GenderDF = pd.concat([pd.DataFrame(index=[sampleGroup1name], data=Gender_sampleGroup1),
                    pd.DataFrame(index=[sampleGroup2name], data=Gender_sampleGroup2)]).fillna(0)
print (GenderDF)

try:
    R_f, p_f = fisher_exact(GenderDF, alternative='two-sided')
    print ('Fisher r,p = ', R_f, p_f)  
except:
    print ('couldnt execute fisher test for gender distributions')

## run for PCI_LVEF_CVA_CABG binary phenotypes

In [ ]:
phen='PCI1_CABG'


with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
    Cardio126_0samples=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
    Cardio126_1samples=pickle.load(fp)
    
sampleGroup1=Cardio126_0samples
sampleGroup1name='%s0samples' %phen
sampleGroup2=Cardio126_1samples
sampleGroup2name='%s1samples' %phen
        
MyPath = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF'   
f1 = '/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TargetDFs/PNP530Cardio126_AgeGender.xlsx' 
PNP530Cardio126_AgeGender = pd.read_excel(f1).set_index('BD')

Age_sampleGroup1 = PNP530Cardio126_AgeGender.loc[sampleGroup1, 'Age'].dropna().tolist()
Age_sampleGroup2 = PNP530Cardio126_AgeGender.loc[sampleGroup2, 'Age'].dropna().tolist()
dataList = [(sampleGroup1name, Age_sampleGroup1), (sampleGroup2name, Age_sampleGroup2)]

try:
    fig, ax = plt.subplots()
    # print (dataList)
    title = '%s vs. %s - Age distribution' % (sampleGroup1name, sampleGroup2name)
    plotHistComprison(dataList, ax, title, showLegend=True, nBins=20, toAnnotate=True, alpha=None, plotType='hist')
    plt.show()
except:
    print ('cant do plots, sorry')

# compare gender distribution:
Gender_sampleGroup1 = Counter(PNP530Cardio126_AgeGender.loc[sampleGroup1, 'Gender_Male'].dropna().tolist())
Gender_sampleGroup2 = Counter(PNP530Cardio126_AgeGender.loc[sampleGroup2, 'Gender_Male'].dropna().tolist())

GenderDF = pd.concat([pd.DataFrame(index=[sampleGroup1name], data=Gender_sampleGroup1),
                    pd.DataFrame(index=[sampleGroup2name], data=Gender_sampleGroup2)]).fillna(0)
print (GenderDF)

try:
    R_f, p_f = fisher_exact(GenderDF, alternative='two-sided')
    print ('Fisher r,p = ', R_f, p_f)  
except:
    print ('couldnt execute fisher test for gender distributions')

In [ ]:
for phen in PCI_LVEF_CVA_CABG.columns.tolist():
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
        Cardio126_0samples=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
        Cardio126_1samples=pickle.load(fp)

        sampleGroup1=Cardio126_0samples
        sampleGroup1name='%s0samples' %phen
        sampleGroup2=Cardio126_1samples
        sampleGroup2name='%s1samples' %phen
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared)


## PCI_cont:

In [ ]:
phen_list=[('PCI',0,2345)]

for item in phen_list:
    phen=item[0]
    group1=str(item[1])
    group2=str(item[2])
    
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s%ssamples' %(phen,group1),'rb') as fp:
        group1data=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s%ssamples' %(phen,group2),'rb') as fp:
        group2data=pickle.load(fp)

        sampleGroup1=group1data
        sampleGroup1name='%s%ssamples' %(phen,group1)
        sampleGroup2=group2data
        sampleGroup2name='%s%ssamples' %(phen,group2)
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25
        percTooMany=0.9

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared,percTooMany)

## other phenotyps:

In [ ]:
phen_list=[('GlucoseDisorder',0,2),('LVEF',0,23),('ostial',0,234)]

for item in phen_list:
    phen=item[0]
    group1=str(item[1])
    group2=str(item[2])
    
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s%ssamples' %(phen,group1),'rb') as fp:
        group1data=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s%ssamples' %(phen,group2),'rb') as fp:
        group2data=pickle.load(fp)

        sampleGroup1=group1data
        sampleGroup1name='%s%ssamples' %(phen,group1)
        sampleGroup2=group2data
        sampleGroup2name='%s%ssamples' %(phen,group2)
        output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
        sampleByClusterDF=sampleByClusterDF
        percShared=0.25
        percTooMany=0.9

        compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared,percTooMany)

In [ ]:
phen='PCI1_CABG'


with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples' %phen,'rb') as fp:
    Cardio126_0samples=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples' %phen,'rb') as fp:
    Cardio126_1samples=pickle.load(fp)
    
sampleGroup1=Cardio126_0samples
sampleGroup1name='%s0samples' %phen
sampleGroup2=Cardio126_1samples
sampleGroup2name='%s1samples' %phen
        
MyPath = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF'   
f1 = '/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TargetDFs/PNP530Cardio126_AgeGender.xlsx' 
PNP530Cardio126_AgeGender = pd.read_excel(f1).set_index('BD')

Age_sampleGroup1 = PNP530Cardio126_AgeGender.loc[sampleGroup1, 'Age'].dropna().tolist()
Age_sampleGroup2 = PNP530Cardio126_AgeGender.loc[sampleGroup2, 'Age'].dropna().tolist()
dataList = [(sampleGroup1name, Age_sampleGroup1), (sampleGroup2name, Age_sampleGroup2)]

try:
    fig, ax = plt.subplots()
    # print (dataList)
    title = '%s vs. %s - Age distribution' % (sampleGroup1name, sampleGroup2name)
    plotHistComprison(dataList, ax, title, showLegend=True, nBins=20, toAnnotate=True, alpha=None, plotType='hist')
    plt.show()
except:
    print ('cant do plots, sorry')

# compare gender distribution:
Gender_sampleGroup1 = Counter(PNP530Cardio126_AgeGender.loc[sampleGroup1, 'Gender_Male'].dropna().tolist())
Gender_sampleGroup2 = Counter(PNP530Cardio126_AgeGender.loc[sampleGroup2, 'Gender_Male'].dropna().tolist())

GenderDF = pd.concat([pd.DataFrame(index=[sampleGroup1name], data=Gender_sampleGroup1),
                    pd.DataFrame(index=[sampleGroup2name], data=Gender_sampleGroup2)]).fillna(0)
print (GenderDF)

try:
    R_f, p_f = fisher_exact(GenderDF, alternative='two-sided')
    print ('Fisher r,p = ', R_f, p_f)  
except:
    print ('couldnt execute fisher test for gender distributions')

## run for paper phenotypes with resampled groupes

### binary:

In [ ]:
phen_list=['ChiefComplaintmapped_anginalpain','DischargeClopidogrel','PCI1_CABG']
# phen_list=['OstialCXDisease','DischargeClopidogrel','PCI1_CABG','MaximalTroponin','ChiefComplaintmapped_anginalpain']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s0samples_reSampled2' %phen,'rb') as fp:
        Cardio126_0samples=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/%s1samples_reSampled2' %phen,'rb') as fp:
        Cardio126_1samples=pickle.load(fp)
        
    sampleGroup1=Cardio126_0samples
    sampleGroup1name='%s0samples_reSampled2' %phen
    sampleGroup2=Cardio126_1samples
    sampleGroup2name='%s1samples_reSampled2' %phen
    output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
    sampleByClusterDF=sampleByClusterDF
    percShared=0.5
    nShared=None

    compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared,nShared=nShared)

### OSTIAL:

In [ ]:
# phen_list=['DischargeClopidogrel','PCI1_CABG','ChiefComplaintmapped_anginalpain']
phen_list=['OstialCXDisease']

for phen in phen_list:
    print ('')
    print (phen)

    # get samples lists:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/ostial0samples_reSampled','rb') as fp:
        Cardio126_0samples=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/ostial234samples_reSampled','rb') as fp:
        Cardio126_1samples=pickle.load(fp)
        
    nShared=math.ceil(min(len(Cardio126_0samples),len(Cardio126_1samples))*0.6)
    print ('nShared=',nShared)

    sampleGroup1=Cardio126_0samples
    sampleGroup1name='%s0samples' %phen
    sampleGroup2=Cardio126_1samples
    sampleGroup2name='%s1samples' %phen
    output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/Fisher_MW_comparisons/'
    sampleByClusterDF=sampleByClusterDF
    percShared=0.25

    compare_seq_clusters_wrapper(sampleByClusterDF,sampleGroup1,sampleGroup1name,sampleGroup2,sampleGroup2name,output_dir,
                                     percShared,nShared=nShared)

# investigate MW-Fisher results:

## def function:

In [62]:
def gen_string_from_dict(dict1):
    a=''  
    for k,v in dict1.items():
        i=':'.join([str(k),str(v)])
        a=','.join([a,i])
        
    return a


def explore_MW_Fisher_results(phen, result_name, output_dir,limitTo=None,result_sig_thresh_p=0.05,
                              resultsToCompare=None,resultsToCompare_sig_thresh_p=0.1,getIdentities=True,sig_clusters_list=None):
    from scipy.stats import chi2_contingency
    #get result df:
#     test_results=[x for x in listdir(results_dir) if ((phen in x) & ('Clusters' in x))]
#     print (len(test_results))
#     if len(test_results)>1: 
#         print ('more than one relevant file')
#         return
#     else: 
#         for result_file in test_results:
#             print ('loading file: ',result_file)
#             result=pd.read_excel(results_dir+result_file
    results_folder=output_dir+'Fisher_MW_comparisons/'
    result=pd.read_excel(results_folder+result_name+'.xlsx')
            
    #get significant results:
    print ('getting significant results:')
    result_sig=result[(result['corr p_values by FDR=0.1_Fisher']<result_sig_thresh_p) | (result['corr p_values by FDR=0.1_MW']<result_sig_thresh_p)]
    
    if limitTo is not None:
        print ('using only top %s results' %limitTo)
        result_sig=result_sig[:limitTo]
        
    #get general sequence info:
    print ('getting general sequence info...')
    from collections import Counter
    
    levDistMatTotalFile=output_dir+'levDistMatTotal.dat'
    levDistMatTotal=pd.read_pickle(levDistMatTotalFile)
    
    #get seqID matrix:
    print ('getting seqID info')
    for f in listdir(output_dir):
        if 'seqID' in f: seq_ID_file=f
    f3=output_dir+seq_ID_file
    seqID=pd.read_pickle(f3)
    
    #get results from other result files:
    if resultsToCompare is not None:
        if sig_clusters_list is None:
            sig_clusters_list=[]
            print ('getting results from other files...')
            for res_name in resultsToCompare:
                if (res_name!=result_name) and ('analyzed' not in res_name):
                    try:
                        res=pd.read_excel(results_folder+res_name)
                    except:
                        res=pd.read_excel(results_folder+res_name+'.xlsx')
                    sig_clusters=res[(res['corr p_values by FDR=0.1_Fisher']<resultsToCompare_sig_thresh_p) | (res['corr p_values by FDR=0.1_MW']<resultsToCompare_sig_thresh_p)]['cluster_head'].tolist()
                    res_name=res_name.replace('Fisher_MW_results_','')
                    sig_clusters_list.append(tuple([res_name,sig_clusters]))
                    print ('number of sig. clusters in %s is %s' %(res_name,len(sig_clusters)))
                
    if getIdentities:
        CDR3_identity=pd.read_excel('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/combined annotation_list_clean_popped.xlsx')
        all_identities= CDR3_identity['combined annotation_list_clean'].dropna().tolist()
        all_indentities_counter=Counter(all_identities)
        print ("all_indentities_counter: ", all_indentities_counter)
        
    #get sequences in cluster and their identities:
    print ('getting info for each cluster...')
    phen_related_identities={};
    
    print ('number of sig results to analyze: ', len(result_sig.index))
    
    for row in result_sig.index:
        print (row)
        cluster_head=result_sig.loc[row,'cluster_head']
        
       
        #get seqID for cluster head:
        try:
            head_ID=list(np.where(seqID == cluster_head)[0])[0]
        except: 
            print ('this row is repeated more than once?')
            head_ID=list(np.where(seqID == cluster_head.tolist()[0])[0])[0]
        #get IDs for all sequences in the cluster:
        cluster_seq_IDs=levDistMatTotal[levDistMatTotal['seq1id']==head_ID]['seq2id'].tolist()

        #get sequences for all sequences in the cluster:
        cluster_seq_sequences=[seqID.loc[x] for x in cluster_seq_IDs]

    #   if get identities:
        if getIdentities:
            annot_list=[]
            for seq in cluster_seq_sequences:
                try:
                    annot=CDR3_identity.loc[seq,'combined annotation_list_clean']
                    try:
                        print (annot.shape) #a way to detect whether a sequence has more than 1 annotation (then it will become a series here)
                        print ('sequence %s has more than 1 annotations' %seq)
                        print (annot)
                        for a1 in list(annot):
                            annot_list.append(a1)
                    except:
                        annot_list.append(annot)
                except:
                    annot_list.append(None)

            identities_count=Counter(annot_list)
            for k,v in identities_count.items():
                try:
                    phen_related_identities[k]=phen_related_identities[k]+v
                except:
                    phen_related_identities[k]=v

            result_sig.loc[row,'n sequences in cluster']=len(cluster_seq_sequences)+1
            result_sig.loc[row,'cluster sequences']=','.join(cluster_seq_sequences)
                        
            a=gen_string_from_dict(identities_count)
            result_sig.loc[row,'annotation summary']=a
           
        if resultsToCompare is not None:
            alsoAppear_list=[]
            for item in sig_clusters_list:
                if (cluster_head in item[1]) & (item[0] not in result_name): alsoAppear_list.append(item[0])
            result_sig.loc[row,'Also sig. in']=','.join(alsoAppear_list)
            
    #compare annotation distributions:
    print ('comparing annotation distributions...')
    phen_related_identities={i:phen_related_identities[i] for i in phen_related_identities if i!=None}
    print ('phen_related_identities:')
    print (phen_related_identities)
    annot_table=pd.concat([pd.DataFrame(index=['all_indentities_counter'],data=all_indentities_counter),
                        pd.DataFrame(index=['phen_related_identities'],data=phen_related_identities)]).fillna(0)
#     print ('annot_table:')
#     print (annot_table)
    result_sig.loc['all_indentities_counter','annotation summary']=gen_string_from_dict(all_indentities_counter)
    result_sig.loc['phen_related_identities','annotation summary']=gen_string_from_dict(phen_related_identities)
       
    try:
        chi2,p,dof,expected = chi2_contingency(annot_table)
        print ('chi2 p = ',p)  
        result_sig.loc['chi_test_p','annotation summary']=p
    except:
        print ('couldnt execute fisher test for this phenotype')
        
    
    print ('saving results...')
    f1=(results_folder+result_name+'_top%sp%s_analyzed.xlsx' %(limitTo,result_sig_thresh_p))
    result_sig.to_excel(f1)
    
    print ('analyzed results saved')
    

    return result_sig
    

## generate sig_clusters_list

In [ ]:
'abc' in 'abcd'

In [63]:
output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/'
results_folder=output_dir+'Fisher_MW_comparisons/'
result_sig_thresh_p=0.05
resultsToCompare_sig_thresh_p=0.1
getIdentities=True
sig_clusters_list=sig_clusters_list

resultsToCompare=[f for f in listdir(output_dir+'Fisher_MW_comparisons/') if isfile(output_dir+'Fisher_MW_comparisons/'+f)]
sig_clusters_list=[]
print ('getting results from other files...')
for res_name in resultsToCompare:
#     if (res_name!=result_name) and ('analyzed' not in res_name):
    if 'analyzed' not in res_name:
        try:
            res=pd.read_excel(results_folder+res_name)
        except:
            res=pd.read_excel(results_folder+res_name+'.xlsx')
        sig_clusters=res[(res['corr p_values by FDR=0.1_Fisher']<resultsToCompare_sig_thresh_p) | (res['corr p_values by FDR=0.1_MW']<resultsToCompare_sig_thresh_p)]['cluster_head'].tolist()
        res_name=res_name.replace('Fisher_MW_results_','')
        sig_clusters_list.append(tuple([res_name,sig_clusters]))
        print ('number of sig. clusters in %s is %s' %(res_name,len(sig_clusters)))

NameError: name 'sig_clusters_list' is not defined

## run for all phenotypes in figure 3:

In [ ]:
phen_list=[('DischargeClopidogrel','Fisher_MW_results_DischargeClopidogrel0samples_DischargeClopidogrel1samples_percShared025_percTooMany095'),
            ('OstialCXDisease','Fisher_MW_results_ostial0samples_ostial234samples_percShared025_percTooMany09'),
            ('MaximalTroponin','Fisher_MW_results_Cardio126_bottom20MaximalTroponin_Cardio126_top20MaximalTroponin_percShared025_percTooMany095'),
            ('PCI1_CABG','Fisher_MW_results_PCI1_CABG0samples_PCI1_CABG1samples_percShared025_percTooMany095'),
            ('ChiefComplaintmapped_anginalpain','Fisher_MW_results_ChiefComplaintmapped_anginalpain0samples_ChiefComplaintmapped_anginalpain1samples_percShared025_percTooMany095')]

for item in phen_list:
    print ('************'+item[0]+'***********')
    print ('')
    phen=item[0]
    result_name=item[1]
    output_dir='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/'
    limitTo=10
    resultsToCompare=[f for f in listdir(output_dir+'Fisher_MW_comparisons/') if isfile(output_dir+'Fisher_MW_comparisons/'+f)]
    # ['Fisher_MW_results_Cardio126_bottom20Age_Cardio126_top20Age_percShared025_percTooMany095',
    # 'Fisher_MW_results_Gender_Male0samples_Gender_Male1samples_percShared025_percTooMany095',
    # 'Fisher_MW_results_Cardio126_bottom20MaximalTroponin_Cardio126_top20MaximalTroponin_percShared025_percTooMany095',
    # '']
    result_sig_thresh_p=0.05
    resultsToCompare_sig_thresh_p=0.1
    getIdentities=True
    sig_clusters_list=sig_clusters_list

    result_sig=explore_MW_Fisher_results(phen, result_name, output_dir,limitTo,result_sig_thresh_p,
                                  resultsToCompare,resultsToCompare_sig_thresh_p,getIdentities,sig_clusters_list)


    

In [ ]:
result_sig

# genertate x matrix with clusters, singelton sequences, annot, top features

## filter sequence matrix for those who are not in any clusters:

In [3]:
# load seqeunce df:
f6 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/prod_sharedBy5-85percPerCohort.dat'
sampleBySeqs_prod_perc=pd.read_pickle(f6)

In [4]:
sampleBySeqs_prod_perc.shape

(656, 6145)

In [23]:
# get all sequences that are in a cluster:

levDistMatTotalProdFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/seqClusters_allProd_maxdist1/levDistMatTotal.dat'
levDistMatTotalProd=pd.read_pickle(levDistMatTotalProdFile)
levDistMatTotalProd.head()

,seq1id,seq2id,dist
0,728000.0,3171972.0,1.0
1,728001.0,1408394.0,1.0
2,728001.0,2027439.0,1.0
3,728002.0,2600393.0,1.0
4,728002.0,3162984.0,1.0


In [28]:
seqs_in_clusters=list(set(levDistMatTotalProd['seq1id'].unique().tolist()+levDistMatTotalProd['seq2id'].unique().tolist()))
print (len(seqs_in_clusters))

3648851


In [27]:
print (levDistMatTotalProd['seq1id'].nunique())
print (levDistMatTotalProd['seq2id'].nunique())

2942935
2940635


In [31]:
f3='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/sharingAnalysis/seqClusters_allProd_maxdist1/seqID_prod.dat'
seqID=pd.read_pickle(f3)
seqID.head()

0        CASSQDLTQGRGYTF
1    CASSYSYYELANSYNEQFF
2        CASSQRTGVSRTQYF
3             CSGKDRSYTF
4         CASVRGRALATAFF
Name: seq, dtype: object

In [32]:
seqs_in_clusters_2=seqID.loc[seqs_in_clusters].tolist()
print (seqs_in_clusters_2[:5])
print (len(seqs_in_clusters_2))

['CASSQDLTQGRGYTF', 'CASSQRTGVSRTQYF', 'CSARLVAETQYF', 'CASSLDGTGIPGELFF', 'CASTPLLAGVYNEQFF']
3648851


In [33]:
print (len(seqID))

5721875


In [34]:
cols_to_include=[x for x in sampleBySeqs_prod_perc.columns.tolist() if x not in seqs_in_clusters_2]
print (len(cols_to_include))

1


In [35]:
sampleBySeqs_prod_perc.columns.tolist()[:5]

['CASSSTSGQYNEQFF',
 'CASSLTVQETQYF',
 'CASSPGTEETQYF',
 'CASSRGTGELFF',
 'CASSLGQGREKLFF']

In [36]:
seqs_in_clusters_2[:5]

['CASSQDLTQGRGYTF',
 'CASSQRTGVSRTQYF',
 'CSARLVAETQYF',
 'CASSLDGTGIPGELFF',
 'CASTPLLAGVYNEQFF']

In [37]:
cols_to_include

['F']

Conclusion: there is no real sequence that is not in clusters among seqeunces that are shared by 5-85 percent of samples!!

## get clsuter df and filter to 0.05-0.85%:

In [16]:
# get clsuter df and filter to 0.05-0.85%:
percTooMany=0.85
sampleByClusterDF=pd.read_pickle('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/\
sharingAnalysis/seqClusters_allProd_maxdist1/sampleByClusterDF_cohortfiltering005-09perc_dropped.dat')

nTooMany = math.ceil(len(sampleByClusterDF) * percTooMany)
print ('filtering out clusters that appear in more than %s (%s) samples' % (nTooMany, percTooMany))
sampleByClusterDF_perc = sampleByClusterDF.loc[:, sampleByClusterDF.count() < nTooMany]
print ('final sampleByClusterDF_perc shape is: ', sampleByClusterDF_perc.shape)

filtering out clusters that appear in more than 558.0 (0.85) samples
final sampleByClusterDF_perc shape is:  (656, 536761)


In [17]:
sampleByClusterDF_perc.count().min()

8

In [19]:
sampleByClusterDF_perc.count().max()

557

In [20]:
sampleByClusterDF.shape

(656, 536984)

In [21]:
sampleByClusterDF_perc.shape

(656, 536761)

In [38]:
f3='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/sharingAnalysis/seqClusters_allProd_maxdist1/sampleByClusterDF_cohortfiltering005-085perc_dropped.dat'
sampleByClusterDF_perc.to_pickle(f3)

## merge sequences and clusters and save:

no need to merge!

## merge clsuters with annot features and top other features:



In [53]:
# get olf X matrix:
f4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender.dat'
X_withPredictedAgeGender=pd.read_pickle(f4)

In [54]:
# get only annot and top features out of it:
annotDF=pd.read_excel('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PNP530Cardio126Combined/\
annotationFeatures/PNP530Cardio126_PNP530Cardio126_annotationFeatures_withRels_noCorr0-99_nanFilled_noConsts.xlsx')

annot_cols=annotDF.columns.tolist()

In [49]:
topFeatures=['AA count_0','AA count_1','NT count_0','NT count_1','cdr3Length_max_0','cdr3Length_max_1','cdr3Length_mean_0',
'cdr3Length_mean_1','cdr3Length_std_0','cdr3Length_std_1','frequencyCount (%)_max_0','frequencyCount (%)_max_1',
'frequencyCount (%)_mean_0','frequencyCount (%)_mean_1','frequencyCount (%)_std_0','frequencyCount (%)_std_1',
'gc_content_0','gc_content_1','max_nt_per_aa_0','max_nt_per_aa_1','mean_nt_per_aa_0','mean_nt_per_aa_1',
'n1Insertion_mean_0','n1Insertion_mean_1','n2Insertion_mean_0','n2Insertion_mean_1','normSeqNums_per2000_NT_0',
'normSeqNums_per2000_NT_1','normSeqNums_per200_AA_0','normSeqNums_per200_AA_1','shannon_aa_0','shannon_aa_1',
'shannon_nt_0','shannon_nt_1','top1000clonal_aa_1','top1000clonal_nt_0','top1000clonal_nt_1','top10clonal_aa_0',
'top10clonal_aa_1','top10clonal_nt_0']


In [55]:
newX_withPredictedAgeGender=X_withPredictedAgeGender[annot_cols+topFeatures]
newX_withPredictedAgeGender.shape

(656, 94)

In [56]:
#merge with clusters and save:
newX_withPredictedAgeGender=pd.merge(newX_withPredictedAgeGender,sampleByClusterDF_perc,how='outer',left_index=True,right_index=True)
newX_withPredictedAgeGender.shape


(656, 536855)

In [58]:
f4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/newX_withPredictedAgeGender.dat'
newX_withPredictedAgeGender.to_pickle(f4)

In [59]:
newX_withPredictedAgeGender_Cardio126=newX_withPredictedAgeGender.loc[Cardio126,:]
newX_withPredictedAgeGender_Cardio126.shape

(126, 536855)

In [60]:
f4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/newX_withPredictedAgeGender_Cardio126.dat'
newX_withPredictedAgeGender_Cardio126.to_pickle(f4)

In [61]:
newX_withPredictedAgeGender_noNans_Cardio126=newX_withPredictedAgeGender_Cardio126.fillna(0)
f5='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/newX_withPredictedAgeGender_noNans_Cardio126.dat'
newX_withPredictedAgeGender_noNans_Cardio126.to_pickle(f5)

# predict isCardio using the new x matrix

# predict isCardio using seqclusters only
1. maybe compare to prediction by non-productive?
2. shap values--> investigate the clusters